Основано на оригинальном ноутбуке от Optinum ([ссылка](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/text_classification_quantization_ort.ipynb)).

Переделано на основе `Quick start`([ссылка](https://huggingface.co/docs/optimum/quicktour#onnx-runtime)).

# Quantizing a model with ONNX Runtime for text classification tasks

This notebook shows how to apply different post-training quantization approaches such as static and dynamic quantization using [ONNX Runtime](https://onnxruntime.ai), for any tasks of the GLUE benchmark. This is made possible thanks to 🤗 [Optimum](https://github.com/huggingface/optimum), an extension of 🤗 [Transformers](https://github.com/huggingface/transformers), providing a set of performance optimization tools enabling maximum efficiency to train and run models on targeted hardwares.  

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers, 🤗 Datasets and 🤗 Optimum. Uncomment the following cell and run it.

In [1]:
!pip install optimum[onnxruntime-gpu]

Make sure your version of 🤗 Optimum is at least 1.1.0 since the functionality was introduced in that version:

In [2]:
from optimum.version import __version__

print(__version__)

1.13.2


In [3]:
task = "sst2"
model_checkpoint = "textattack/bert-base-uncased-SST-2"

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the dataset and get the metric we need to use for evaluation. This can be easily done with the functions `load_dataset` and `load_metric`.  

In [4]:
from datasets import load_dataset, load_metric

`load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [5]:
actual_task = "mnli" if task == "mnli-mm" else task
validation_split = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
eval_dataset = load_dataset("glue", actual_task, split=validation_split)
metric = load_metric("glue", actual_task)

<ipython-input-5-7ea79d66af03>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", actual_task)


Note that `load_metric` has loaded the proper metric associated to your task, which is:

- for CoLA: [Matthews Correlation Coefficient](https://en.wikipedia.org/wiki/Matthews_correlation_coefficient)
- for MNLI (matched or mismatched): Accuracy
- for MRPC: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for QNLI: Accuracy
- for QQP: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for RTE: Accuracy
- for SST-2: Accuracy
- for STS-B: [Pearson Correlation Coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) and [Spearman's_Rank_Correlation_Coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)
- for WNLI: Accuracy

so the metric object only computes the one(s) needed for your task.

## Preprocessing the data

To preprocess our dataset, we will need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [6]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer than what the model selected can handle will be truncated to the maximum length accepted by the model.

In [7]:
sentence1_key, sentence2_key = task_to_keys[task]

def preprocess_function(examples, tokenizer):
    args = (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
    return tokenizer(*args, padding="max_length", max_length=128, truncation=True)

## Applying quantization on the model

We can set our `quantization_approach` to either `dynamic` or `static` in order to apply respectively dynamic and static quantization.
- Post-training static quantization : introduces an additional calibration step where data is fed through the network in order to compute the activations quantization parameters.
- Post-training dynamic quantization : dynamically computes activations quantization parameters based on the data observed at runtime.

In [8]:
QUANTIZATION_APPROACH = ["dynamic", "static"]

quantization_approach = "static"

First, let's create the output directory where the resulting quantized model will be saved.

In [9]:
import os

model_name = model_checkpoint.split("/")[-1]
output_dir = f"{model_name}-{quantization_approach}-quantization"
save_dir = f"{model_name}"
os.makedirs(output_dir, exist_ok=True)

We will use the [🤗 Optimum](https://github.com/huggingface/optimum) library to instantiate an `ORTQuantizer`, which will take care of the quantization process. To instantiate an `ORTQuantizer`, we need to provide a path to a converted ONNX checkpoint or instance of a `ORTModelForXXX`.

In [10]:
# load model and save with tokenizer
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTQuantizer, ORTModelForSequenceClassification

# Loading Model from the Hub and convert to ONNX
ort_model = ORTModelForSequenceClassification.from_pretrained(model_checkpoint, export=True)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Save the ONNX model and tokenizer
ort_model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

# Create a quantizer from a ORTModelForXXX
quantizer = ORTQuantizer.from_pretrained(ort_model)


Framework not specified. Using pt to export to ONNX.
Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.1.0+cu118
Overriding 1 configuration item(s)
	- use_cache -> False


We also need to create an `QuantizationConfig` instance, which is the configuration handling the ONNX Runtime quantization related parameters.

* We set `per_channel` to `False` in order to apply per-tensor quantization on the weights. As opposed to per-channel quantization, which introduces one set of quantization parameters per channel, per-tensor quantization means that there will be one set of quantization parameters per tensor.
* We set the number of samples `num_calibration_samples` to use for the calibration step resulting from static quantization to `40`.
* `operators_to_quantize` is used to specify the types of operators to quantize, here we want to quantize all the network's fully connected and embedding layers.

In [11]:
from optimum.onnxruntime.configuration import QuantizationConfig, AutoCalibrationConfig
from optimum.onnxruntime.quantization import QuantFormat, QuantizationMode, QuantType

per_channel = False
num_calibration_samples = 40
operators_to_quantize = ["MatMul", "Add", "Gather"]
apply_static_quantization = quantization_approach == "static"

qconfig = QuantizationConfig(
    is_static=apply_static_quantization,
    format=QuantFormat.QDQ if apply_static_quantization else QuantFormat.QOperator,
    mode=QuantizationMode.QLinearOps if apply_static_quantization else QuantizationMode.IntegerOps,
    activations_dtype=QuantType.QInt8 if apply_static_quantization else QuantType.QUInt8,
    weights_dtype=QuantType.QInt8,
    per_channel=per_channel,
    operators_to_quantize=operators_to_quantize,
)

When applying static quantization, we need to perform a calibration step where the activations quantization ranges are computed. This additionnal step should only be performed in the case of static quantization and not for dynamic quantization.
Because the quantization of certain nodes often results in degradation in accuracy, we create an instance of `QuantizationPreprocessor` to determine the nodes to exclude when applying static quantization.

In [12]:
from functools import partial
from transformers import AutoTokenizer
from optimum.onnxruntime.preprocessors import QuantizationPreprocessor
from optimum.onnxruntime.preprocessors.passes import (
    ExcludeGeLUNodes,
    ExcludeLayerNormNodes,
    ExcludeNodeAfter,
    ExcludeNodeFollowedBy,
)

# Load tokenizer for preprocessing
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

ranges = None
quantization_preprocessor = None
if apply_static_quantization:
    # Create the calibration dataset used for the calibration step
    calibration_dataset = quantizer.get_calibration_dataset(
        "glue",
        dataset_config_name=actual_task,
        preprocess_function=partial(preprocess_function, tokenizer=tokenizer),
        num_samples=num_calibration_samples,
        dataset_split="train",
    )
    calibration_config = AutoCalibrationConfig.minmax(calibration_dataset)
    # Perform the calibration step: computes the activations quantization ranges
    ranges = quantizer.fit(
        dataset=calibration_dataset,
        calibration_config=calibration_config,
        operators_to_quantize=qconfig.operators_to_quantize,
    )
    quantization_preprocessor = QuantizationPreprocessor()
    # Exclude the nodes constituting LayerNorm
    quantization_preprocessor.register_pass(ExcludeLayerNormNodes())
    # Exclude the nodes constituting GELU
    quantization_preprocessor.register_pass(ExcludeGeLUNodes())
    # Exclude the residual connection Add nodes
    quantization_preprocessor.register_pass(ExcludeNodeAfter("Add", "Add"))
    # Exclude the Add nodes following the Gather operator
    quantization_preprocessor.register_pass(ExcludeNodeAfter("Gather", "Add"))
    # Exclude the Add nodes followed by the Softmax operator
    quantization_preprocessor.register_pass(ExcludeNodeFollowedBy("Add", "Softmax"))

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2089: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=False' instead.
  warnings.warn(
Using static quantization schema (dataset: glue, method: CalibrationMethod.MinMax)
Creating calibrator: CalibrationMethod.MinMax(CalibrationConfig(dataset_name='glue', dataset_config_name='sst2', dataset_split='train', dataset_num_samples=40, method=<CalibrationMethod.MinMax: 0>, num_bins=None, num_quantized_bins=None, percentile=None, moving_average=False, averaging_constant=0.01))
Computing calibration ranges


Finally, we export the quantized model.

In [13]:
quantizer.quantize(
    save_dir=output_dir,
    calibration_tensors_range=ranges,
    quantization_config=qconfig,
    preprocessor=quantization_preprocessor,
)

Creating static quantizer: QDQ (mode: QLinearOps, schema: s8/s8, channel-wise: False)
Preprocessor detected, collecting nodes to include/exclude
Quantizing model...
Saving quantized model at: bert-base-uncased-SST-2-static-quantization (external data format: False)
Configuration saved in bert-base-uncased-SST-2-static-quantization/ort_config.json


PosixPath('bert-base-uncased-SST-2-static-quantization')

## Evaluation

To evaluate our resulting quantized model we need to define how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B).

The metric chosen to evaluate the quantized model's performance will be Matthews correlation coefficient (MCC) for CoLA, Pearson correlation coefficient (PCC) for STS-B and accuracy for any other tasks.

In [14]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

Then to apply the preprocessing on all the sentences (or pairs of sentences) of our validation dataset, we just use the `map` method of our `dataset` object that was earlier created. This will apply the `preprocess_function` function on all the elements of our validation dataset.

In [15]:
eval_dataset = eval_dataset.map(partial(preprocess_function, tokenizer=tokenizer), batched=True)

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Finally, to estimate the drop in performance resulting from quantization, we are going to perform an evaluation step for both models (before and after quantization). In order to perform the latter, we will need to instantiate an `ORTModel` and thus need:

* The path of the model to evaluate.
* The dataset to use for the evaluation step.
* The model's ONNX configuration `onnx_config` associated to the model. This instance of  `OnnxConfig` describes how to export the model through the ONNX format.
* The function that will be used to compute the evaluation metrics `compute_metrics` that was defined previously.

In [16]:
# оставим подмножество датасета из 200 примеров
eval_dataset = eval_dataset.select(range(200))

# Выполним предсказания исходной моделью

In [30]:
from time import perf_counter

In [31]:
# Создадим исходную модель из ONNX-файла
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import pipeline, AutoTokenizer

save_directory='bert-base-uncased-SST-2'
file_name='model.onnx'

model = ORTModelForSequenceClassification.from_pretrained(save_directory, file_name=file_name)
tokenizer = AutoTokenizer.from_pretrained(save_directory)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [32]:
# функция для предсказания результата
from sklearn.metrics import accuracy_score

map_classes = {'LABEL_0': 0,
               'LABEL_1': 1}

def make_prediction(classifier, eval_dataset, map_classes):
    results = classifier(eval_dataset['sentence'])
    pred_labels = [map_classes[result['label']] for result in results]
    return pred_labels

In [33]:
# засекаем время инференса
start_time = perf_counter()

# предсказываем класс исходной моделью
pred_labels = make_prediction(classifier, eval_dataset, map_classes)

end_time = perf_counter()
print(f"Время инференса {(end_time - start_time):.2f} секунд")

Время инференса 26.16 секунд


In [34]:
# оцениваем точность исходной модели
pure_model_acc = accuracy_score(eval_dataset['label'], pred_labels)
print(f'Acuracy исходной модели {pure_model_acc}')

Acuracy исходной модели 0.915


# Выполним предсказания моделью после статической квантизации

In [35]:
# Создадим исходную модель из ONNX-файла
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import pipeline, AutoTokenizer

save_directory='bert-base-uncased-SST-2-static-quantization'
file_name='model_quantized.onnx'

model = ORTModelForSequenceClassification.from_pretrained(save_directory, file_name=file_name)
tokenizer = AutoTokenizer.from_pretrained(save_directory)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [37]:
# засекаем время инференса
start_time = perf_counter()

# предсказываем класс квантизованной моделью
pred_labels = make_prediction(classifier, eval_dataset, map_classes)

end_time = perf_counter()
print(f"Время инференса {(end_time - start_time):.2f} секунд")

Время инференса 26.78 секунд


In [24]:
# оцениваем точность квантизованной модели
pure_model_acc = accuracy_score(eval_dataset['label'], pred_labels)
print(f'Acuracy квантизованной модели {pure_model_acc}')

Acuracy квантизованной модели 0.89


# Сраним размеры ONNX-файлов

In [25]:
model_onnx_path = '/content/bert-base-uncased-SST-2/model.onnx'
model_size = os.path.getsize(model_onnx_path) / (1024*1024)

print(f'Размер исходной модели {model_size:.2f} Мб')

Размер исходной модели 417.90 Мб


In [26]:
model_onnx_quant_path = '/content/bert-base-uncased-SST-2-static-quantization/model_quantized.onnx'
model_quant_size = os.path.getsize(model_onnx_quant_path) / (1024*1024)

print(f'Размер квантизованной модели {model_quant_size:.2f} Мб')

Размер квантизованной модели 106.85 Мб


Отношение размеров моделей:

In [27]:
# выигрыш по размеру файла в 4 раза!
round(model_size / model_quant_size, 2)

3.91

# Запуск скрипта квантизации

In [28]:
!python run_glue.py \
    --model_name_or_path distilbert-base-uncased-finetuned-sst-2-english \
    --task_name sst2 \
    --optimization_level 1 \
    --do_eval \
    --output_dir /content/optimized_distilbert_sst2

python3: can't open file '/content/run_glue.py': [Errno 2] No such file or directory
